In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [54]:
import importlib
importlib.reload(utils)

<module 'utils' from '/home/gwiedemann/microNER/scripts/utils.py'>

In [55]:
trainSentences = utils.modify_labels(utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv'))
devSentences = utils.modify_labels(utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv'))
testSentences = utils.modify_labels(utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv'))

trainSentences.extend(utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio'))
devSentences.extend(utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio'))
testSentences.extend(utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio'))

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


36152
5067
8105


In [56]:
print(testSentences[3900])

[['Auch', 'O'], ['seine', 'O'], ['Teamkollegen', 'O'], ['sind', 'O'], ['zuversichtlich', 'O'], [',', 'O'], ['dass', 'O'], ['die', 'O'], ['Weberruß-Truppe', 'O'], ['bereits', 'O'], ['morgen', 'O'], ['alles', 'O'], ['klar', 'O'], ['macht', 'O'], ['.', 'O'], ['30:29', 'O'], ['lautet', 'O'], ['der', 'O'], ['Tipp', 'O'], ['von', 'O'], ['Tim', 'B-PER'], ['Dittrich', 'I-PER'], [',', 'O'], ['34:32', 'O'], ['der', 'O'], ['von', 'O'], ['Nico', 'B-PER'], ['Weißmann', 'I-PER'], ['.', 'O']]


In [57]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [58]:
print(len(labelSet))
print(models.max_sequence_length)

9
100


In [59]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [60]:
print(models.label2Idx)

{'PADDING_TOKEN': 0, 'B-MISC': 1, 'B-LOC': 6, 'B-PER': 7, 'I-MISC': 8, 'I-LOC': 9, 'I-ORG': 2, 'O': 3, 'I-PER': 5, 'B-ORG': 4}


In [61]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [62]:
print(models.case2Idx)

{'other': 5, 'PADDING_TOKEN': 0, 'allUpper': 3, 'contains_digit': 7, 'numeric': 1, 'mainly_numeric': 6, 'allLower': 2, 'initialUpper': 4}


In [63]:
print(type(trainSentences))

<class 'list'>


In [66]:
print(trainSentences[11])

[['Scenes', 'B-MISC'], ['of', 'I-MISC'], ['a', 'I-MISC'], ['Sexual', 'I-MISC'], ['Nature', 'I-MISC'], ['(', 'O'], ['GB', 'O'], ['2006', 'O'], [')', 'O'], ['-', 'O'], ['Regie', 'O'], [':', 'O'], ['Ed', 'B-PER'], ['Blum', 'I-PER'], ['Shortbus', 'B-MISC'], ['(', 'O'], ['USA', 'B-LOC'], ['2006', 'O'], [')', 'O'], ['-', 'O'], ['Regie', 'O'], [':', 'O'], ['John', 'B-PER'], ['Cameron', 'I-PER'], ['Mitchell', 'I-PER'], [':Film', 'O'], ['über', 'O'], ['den', 'O'], ['gleichnamigen', 'O'], ['New', 'B-MISC'], ['Yorker', 'I-MISC'], ['Club', 'O'], [',', 'O'], ['der', 'O'], ['verschiedensten', 'O'], ['Paaren', 'O'], ['eine', 'O'], ['Plattform', 'O'], ['zur', 'O'], ['Aufarbeitung', 'O'], ['ihrer', 'O'], ['Probleme', 'O'], ['bietet', 'O'], ['.', 'O']]


In [67]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'4': 5, 'É': 6, 'f': 7, '6': 8, 'İ': 9, 'X': 11, 'ő': 12, '⋅': 13, 'У': 15, 'è': 290, 'ι': 254, 'ż': 18, 'ı': 19, 'K': 20, 'ì': 10, 'й': 22, 'Š': 23, 'ь': 24, 'Ł': 239, 'T': 25, 'o': 26, '≘': 28, 'ā': 29, 'ψ': 30, 'O': 31, '€': 238, 'V': 33, '章': 181, '·': 34, 'č': 36, '\x9a': 180, '\xad': 37, '→': 39, 'Œ': 40, 'r': 41, 'ñ': 42, 'ο': 43, 'τ': 61, 'ū': 44, '*': 306, 'Å': 45, '妃': 49, 'ж': 47, '殿': 48, 'α': 50, 'e': 51, 'ø': 112, 'î': 267, 'β': 201, 'a': 53, 'λ': 55, '造': 296, '樓': 56, 'N': 57, 'm': 185, 'á': 58, '.': 59, '貴': 63, 'ī': 62, 'д': 314, '術': 64, 'Ż': 66, 'w': 67, '¤': 68, 'ğ': 261, '\x95': 72, 'И': 74, '佐': 73, '守': 81, 'F': 329, 'у': 79, 'П': 78, '„': 80, 'л': 82, 'Î': 83, 'µ': 84, 'ł': 85, 'ŏ': 21, 'М': 86, 'ḳ': 87, '</W>': 4, 'Z': 88, 'Ü': 89, 'g': 291, 'ラ': 90, 's': 91, '台': 92, '«': 93, '−': 94, '鶴': 14, 'Č': 95, 'и': 96, 'є': 99, '…': 250, 'г': 100, 'Â': 101, 'UNKNOWN': 335, '太': 102, '`': 103, 'ý': 76, 'а': 126, ':': 104, 'ň': 105, 'Y': 106, 'ε': 107, 'С': 69, '\u200

In [68]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [50]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [51]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

10
10


# Test Model

In [18]:
# import importlib
# importlib.reload(models)

In [69]:
tmp_model_filename = 'model_lstm_germeval-conll_v2.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_lstm_v2()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10752       char_input[0][0]                 
__________________________________________________________________________________________________
words_input (InputLayer)        (None, None, 300)    0                                            
__________________________________________________________________________________________________
case_embed

In [70]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [history]
)

Epoch 1/10
2259/2260 [============================>.] - ETA: 0s - loss: 0.0190 - acc: 0.9925
New maximum F1 score: 0.7921085464753587 (before: 0) Saving to model_lstm_germeval-conll_v2.h5
2260/2260 [==============================] - 1445s 640ms/step - loss: 0.0190 - acc: 0.9925 - val_loss: -0.0010 - val_acc: 0.9942
Epoch 2/10
2259/2260 [============================>.] - ETA: 0s - loss: -0.0214 - acc: 0.9950
New maximum F1 score: 0.805161485058859 (before: 0.7921085464753587) Saving to model_lstm_germeval-conll_v2.h5
2260/2260 [==============================] - 1440s 637ms/step - loss: -0.0214 - acc: 0.9950 - val_loss: -0.0405 - val_acc: 0.9946
Epoch 3/10
2259/2260 [============================>.] - ETA: 0s - loss: -0.0653 - acc: 0.9955
New maximum F1 score: 0.8181749793124202 (before: 0.805161485058859) Saving to model_lstm_germeval-conll_v2.h5
2260/2260 [==============================] - 1437s 636ms/step - loss: -0.0653 - acc: 0.9955 - val_loss: -0.0855 - val_acc: 0.9947
Epoch 4/10
22

In [71]:
print(history.acc)
print(history.f1_scores)

[0.9942056472093979, 0.9945766689722322, 0.9947483783669789, 0.9951371623727728, 0.9946575875180653, 0.9952003105236789, 0.9951036053106633, 0.9952161090012154, 0.9949358573485955, 0.9955298930247766]
[0.7921085464753587, 0.805161485058859, 0.8181749793124202, 0.8268345102855388, 0.8130154445943919, 0.8285973397823457, 0.82606728013275, 0.8304675716440422, 0.8234937016239186, 0.840721491898502]


In [72]:
model.load_weights(tmp_model_filename)

In [73]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [history]
)

Epoch 1/10
142/142 [==============================] - 221s 2s/step - loss: -0.4092 - acc: 0.9971 - val_loss: -0.4044 - val_acc: 0.9953
Epoch 2/10
142/142 [==============================] - 199s 1s/step - loss: -0.4122 - acc: 0.9972 - val_loss: -0.4074 - val_acc: 0.9953
Epoch 3/10
142/142 [==============================] - 197s 1s/step - loss: -0.4149 - acc: 0.9971 - val_loss: -0.4097 - val_acc: 0.9952
Epoch 4/10
142/142 [==============================] - 198s 1s/step - loss: -0.4175 - acc: 0.9970 - val_loss: -0.4127 - val_acc: 0.9952
Epoch 5/10
142/142 [==============================] - 199s 1s/step - loss: -0.4201 - acc: 0.9969 - val_loss: -0.4156 - val_acc: 0.9952
Epoch 6/10
142/142 [==============================] - 196s 1s/step - loss: -0.4230 - acc: 0.9969 - val_loss: -0.4184 - val_acc: 0.9953
Epoch 7/10
142/142 [==============================] - 195s 1s/step - loss: -0.4260 - acc: 0.9970 - val_loss: -0.4211 - val_acc: 0.9953
Epoch 8/10
142/142 [==============================] - 2

In [74]:
model.load_weights(tmp_model_filename)

In [76]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

(0.8388566222525908, 0.8312831508859045, 0.8350527151116655)


In [77]:
f = open('germeval-conll_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = " ".join([tok[0], correctlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close

<function TextIOWrapper.close()>